# Imports (compatible for Python 2 and Python 3)

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [3]:
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.994811,NaN,801,2/1/2015 0:00,2/1/2015 0:14,521,8 Ave & W 31 St,40.750450,1,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978.0,2
1,NaN,-73.985649,NaN,245,2/1/2015 1:03,2/1/2015 1:07,438,St Marks Pl & 1 Ave,40.727791,89,297,E 15 St & 3 Ave,40.734232,-73.986923,20142,Subscriber,1990.0,1
2,NaN,-73.990093,NaN,379,2/1/2015 0:00,2/1/2015 0:07,497,E 17 St & Broadway,40.737050,2,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993.0,1
3,NaN,-73.973715,NaN,2474,2/1/2015 0:01,2/1/2015 0:42,281,Grand Army Plaza & Central Park S,40.764397,3,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969.0,2
4,NaN,-74.004704,NaN,818,2/1/2015 0:01,2/1/2015 0:15,2004,6 Ave & Broome St,40.724399,4,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985.0,2
